<a href="https://colab.research.google.com/github/NancyBiyahut/face-acne-grding-using-CNN/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import models , layers
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE_l= 550
IMAGE_SIZE_h= 550
BATCH_SIZE = 32
CHANELS = 3
EPOCHS = 5

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/db",
    shuffle = True,
    image_size = (IMAGE_SIZE_l,IMAGE_SIZE_h) ,
    batch_size = BATCH_SIZE
)

Found 999 files belonging to 3 classes.


In [4]:
class_names = dataset.class_names
class_names

['Level_0', 'Level_1', 'Level_2']

In [5]:
for image_batch , label_batch in dataset.take(1):
 print(image_batch[0].shape)

(550, 550, 3)


In [6]:
len(dataset)

32

In [7]:
train_size = 0.8
len(dataset) * train_size

25.6

In [8]:
train_ds= dataset.take(25)
test_ds = dataset.skip(25)

In [9]:
val_size = 0.1
len(dataset) * val_size

3.2

In [10]:
val_ds = test_ds.take(3)
len(val_ds)

3

In [11]:
test_ds = test_ds.skip(3)
len(test_ds)

4

In [12]:
train_ds= train_ds.cache().shuffle(500).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds= test_ds.cache().shuffle(500).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds= val_ds.cache().shuffle(500).prefetch(buffer_size = tf.data.AUTOTUNE)

In [13]:
resize_and_rescale = tf.keras.Sequential([
      layers.experimental.preprocessing.Resizing(IMAGE_SIZE_l,IMAGE_SIZE_h),                
     layers.experimental.preprocessing.Rescaling(1.0/550)               
])

In [14]:
data_augumentation = tf.keras.Sequential([
     layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
      layers.experimental.preprocessing.RandomRotation(0.2)                
])

In [15]:
input_shape = (BATCH_SIZE, IMAGE_SIZE_l , IMAGE_SIZE_h , CHANELS)
n_classes = 3
model = models.Sequential([
  resize_and_rescale,
  data_augumentation,
  layers.Conv2D(32,(3,3),activation='relu' , input_shape= input_shape),
  layers.MaxPooling2D((2,2)) ,
  layers.Conv2D(64,(3,3),activation='relu' , input_shape= input_shape),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(64,(3,3),activation='relu' , input_shape= input_shape),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(64,(3,3),activation='relu' , input_shape= input_shape),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(64,(3,3),activation='relu' , input_shape= input_shape),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(64,(3,3),activation='relu' , input_shape= input_shape),
  layers.MaxPooling2D((2,2)),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense( n_classes, activation= 'softmax')

])

model.build(input_shape=input_shape)

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 550, 550, 3)         0         
                                                                 
 sequential_1 (Sequential)   (32, 550, 550, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 548, 548, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 274, 274, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 272, 272, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 136, 136, 64)       0         
 2D)                                                  

In [17]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
from tensorflow.python.ops.gen_batch_ops import batch
history = model.fit(
   train_ds,
   epochs = EPOCHS,
   batch_size = BATCH_SIZE ,
   verbose = 1,
   validation_data = val_ds
)

Epoch 1/5
10/25 [===========>..................] - ETA: 4:11 - loss: 1.0368 - accuracy: 0.3656

In [ ]:
score = model.evaluate(test_ds)